In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/similar_logic/instr_reg/v0/without_head/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/similar_logic/instr_reg/v0/without_head/graph.jsonl'

In [3]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [4]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [5]:
conv[0]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk_high', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'clk_out1', 'type': 'output port'}, {'id': 3, 'content': 'clk_out2', 'type': 'output port'}, {'id': 4, 'content': 'clk_out3', 'type': 'output port'}, {'id': 5, 'content': 'div1', 'type': 'submodule'}, {'id': 6, 'content': 'div2', 'type': 'submodule'}, {'id': 7, 'content': 'div3', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[1, 0, 1, 0, 1, 0, 5, 6, 7], [5, 5, 6, 6, 7, 7, 2, 3, 4]]\n, A digital clock divider module designed to generate multiple lower frequency clocks from a single high-frequency input clock. It features various output clocks derived from input frequencies using independent submodules for frequency division.\n\nModule name:\n    clock_divider\n\nInput ports:\n    clk_high: High-frequency clock input for the divider.\n    rst: Active low reset signal to initialize all 

In [6]:
conv[0]['conversations'][1]['value']

'module clock_divider (\n    input clk_high,\n    input rst,\n    output clk_out1,\n    output clk_out2,\n    output clk_out3\n);\n\n    // Submodule for dividing clock by 2\n    wire clk_div2;\n    clock_div_2 div1 (\n        .clk_high(clk_high),\n        .rst(rst),\n        .clk_out(clk_div2)\n    );\n\n    // Submodule for dividing clock by 4\n    wire clk_div4;\n    clock_div_4 div2 (\n        .clk_high(clk_high),\n        .rst(rst),\n        .clk_out(clk_div4)\n    );\n\n    // Submodule for dividing clock by 8\n    wire clk_div8;\n    clock_div_8 div3 (\n        .clk_high(clk_high),\n        .rst(rst),\n        .clk_out(clk_div8)\n    );\n\n    // Output assignments\n    assign clk_out1 = clk_div2;\n    assign clk_out2 = clk_div4;\n    assign clk_out3 = clk_div8;\n\nendmodule\n\n// Submodule for dividing the clock by 2\nmodule clock_div_2 (\n    input clk_high,\n    input rst,\n    output reg clk_out\n);\n\n    always @(posedge clk_high or negedge rst) begin\n        if (!rst) be

In [7]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk_high', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'clk_out1', 'type': 'output port'}, {'id': 3, 'content': 'clk_out2', 'type': 'output port'}, {'id': 4, 'content': 'clk_out3', 'type': 'output port'}, {'id': 5, 'content': 'div1', 'type': 'submodule'}, {'id': 6, 'content': 'div2', 'type': 'submodule'}, {'id': 7, 'content': 'div3', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[1, 0, 1, 0, 1, 0, 5, 6, 7], [5, 5, 6, 6, 7, 7, 2, 3, 4]]\n, A digital clock divider module designed to generate multiple lower frequency clocks from a single high-frequency input clock. It features various output clocks derived from input frequencies using independent submodules for frequency division.\n\nModule name:\n    clock_divider\n\nInput ports:\n    clk_high: High-frequency clock input for the divider.\n    rst: Active low reset signal to initialize all 

In [8]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + '\n' + module_head

In [9]:
# conv[100]['conversations'][0]['value']

In [10]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/similar_logic/instr_reg/v0/with_head/conversations.json'



In [11]:
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)

In [12]:
import json
with open(new_conv_path, 'r') as f:
    new_conv = json.load(f)
    


In [13]:
len(new_conv)

480

In [ ]:
new_conv[100]['conversations'][0]['value']

In [32]:
len(graph)

2148